# 03 - Fraud Detection Engine
Run all five detectors against the transaction data and compute unified risk scores.

In [ ]:
import sys
sys.path.insert(0, "..")
import pandas as pd
from pathlib import Path
from src.detectors import ALL_DETECTORS
from src.detectors.structuring import StructuringDetector
from src.detectors.account_takeover import AccountTakeoverDetector
from src.detectors.kiting import KitingDetector
from src.detectors.dormant import DormantAccountDetector
from src.detectors.anomaly import AnomalyDetector
from src.scoring import compute_composite_score
from src.features import build_feature_matrix

In [ ]:
data_dir = Path("../data/processed")
parquet_files = list(data_dir.glob("*.parquet"))
datasets = {f.stem: pd.read_parquet(f) for f in parquet_files}
print(f"Loaded: {list(datasets.keys())}")

# Identify main transaction dataset
txn_key = next((k for k in datasets if "trans" in k.lower()), list(datasets.keys())[0] if datasets else None)
txn = datasets[txn_key] if txn_key else pd.DataFrame()
print(f"Using '{txn_key}' as primary transaction data: {txn.shape}")

## Run Individual Detectors

In [ ]:
print("=" * 50)
print("STRUCTURING DETECTOR")
print("=" * 50)
sd = StructuringDetector()
structuring_results = sd.detect(txn)
print(f"Flagged accounts: {len(structuring_results[structuring_results['risk_score'] > 0])}")
structuring_results.sort_values("risk_score", ascending=False).head(10)

In [ ]:
print("=" * 50)
print("ACCOUNT TAKEOVER DETECTOR")
print("=" * 50)
ato = AccountTakeoverDetector()
ato_results = ato.detect(txn)
print(f"Flagged accounts: {len(ato_results[ato_results['risk_score'] > 0])}")
ato_results.sort_values("risk_score", ascending=False).head(10)

In [ ]:
print("=" * 50)
print("KITING DETECTOR")
print("=" * 50)
kd = KitingDetector()
kiting_results = kd.detect(txn)
print(f"Flagged accounts: {len(kiting_results[kiting_results['risk_score'] > 0])}")
kiting_results.sort_values("risk_score", ascending=False).head(10)

In [ ]:
print("=" * 50)
print("DORMANT ACCOUNT DETECTOR")
print("=" * 50)
dd = DormantAccountDetector()
dormant_results = dd.detect(txn)
print(f"Flagged accounts: {len(dormant_results[dormant_results['risk_score'] > 0])}")
dormant_results.sort_values("risk_score", ascending=False).head(10)

In [ ]:
print("=" * 50)
print("ANOMALY DETECTOR (Isolation Forest)")
print("=" * 50)
ad = AnomalyDetector()
anomaly_results = ad.detect(txn)
print(f"Flagged accounts: {len(anomaly_results[anomaly_results['risk_score'] > 0])}")
anomaly_results.sort_values("risk_score", ascending=False).head(10)

## Composite Risk Scores

In [ ]:
detector_results = {
    "structuring": structuring_results,
    "account_takeover": ato_results,
    "kiting": kiting_results,
    "dormant": dormant_results,
    "anomaly": anomaly_results,
}
# Filter to detectors that returned results
detector_results = {k: v for k, v in detector_results.items() if len(v) > 0}

composite = compute_composite_score(detector_results)
print(f"\nRisk tier distribution:")
print(composite["risk_tier"].value_counts())
print(f"\nTop 20 highest-risk accounts:")
composite.head(20)

## Save Results

In [ ]:
output_dir = Path("../output")
output_dir.mkdir(parents=True, exist_ok=True)
composite.to_csv(output_dir / "risk_scores.csv", index=False)
for name, result in detector_results.items():
    result.to_csv(output_dir / f"detector_{name}.csv", index=False)
print("Results saved to output/")